In [1]:
import os
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

import polars as pl
import librosa
import librosa.display
import matplotlib.pyplot as plt
import helpers.input_processor as ip
import tqdm
import tensorflow as tf
from tensorflow.data import Dataset as tfds
import tensorflow_io as tfio
import math, random
import numpy as np

from matplotlib import pyplot as plt
import tensorflow.python.util as util

ModuleNotFoundError: No module named 'polars'

In [ ]:
import torch
import torchaudio
import polars as pl
import matplotlib.pyplot as plt
import os
import numpy as np
import tqdm
from torchaudio import transforms
from IPython.display import Audio
import math, random

import helpers.input_processor as ip

: 

In [ ]:
from datetime import datetime
from packaging import version
# pip install -U tensorboard_plugin_profile

: 

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

: 

In [ ]:
def balanceData(df, method):
    if method==1:
        #balance the data so that there is an equal number of murmur positive and murmur negative samples
        #do this by duplicating random rows of whichever group (pos or neg) is smaller
        neg_df = df.filter(pl.col(target_label)==0.0)
        pos_df = df.filter(pl.col(target_label)==1.0)
        numNeg = neg_df.height
        numPos = pos_df.height

        while numNeg != numPos:
            if numNeg < numPos:
                df.vstack(neg_df.sample(n=min(numPos-numNeg, neg_df.height), shuffle=True), in_place=True)
            else: 
                df.vstack(pos_df.sample(n=min(numNeg-numPos, pos_df.height), shuffle=True), in_place=True)
            numNeg = df.filter(pl.col(target_label)==0.0).height
            numPos = df.filter(pl.col(target_label)==1.0).height

        #reshuffle rows
        df = df.sample(frac=1.0, shuffle=True)

        #check number of positive and negative samples
        numNeg = df.filter(pl.col(target_label)==0.0).height
        numPos = df.filter(pl.col(target_label)==1.0).height
        print('Total Samples:       ', df.height)
        print('Positive Samples:    ', numPos)
        print('Negative Samples:    ', numNeg)
        print('Percent Positive Samples:    ', numPos/(numPos+numNeg))
    elif method==2:
        #balance the data so that there is an equal number of murmur positive and murmur negative samples
        #do this by filtering out rows of whichever group (pos or neg) is larger
        neg_df = df.filter(pl.col(target_label)==0.0)
        pos_df = df.filter(pl.col(target_label)==1.0)
        numNeg = neg_df.height
        numPos = pos_df.height

        if numNeg < numPos:
            df = neg_df.vstack(pos_df.sample(n=numNeg))
        elif numPos < numNeg:
            df = pos_df.vstack(neg_df.sample(n=numPos))
        else:
            df = neg_df.vstack(pos_df)

        #reshuffle rows
        df = df.sample(frac=1.0, shuffle=True)

        #check number of positive and negative samples
        numNeg = df.filter(pl.col(target_label)==0.0).height
        numPos = df.filter(pl.col(target_label)==1.0).height
        print('Total Samples:       ', df.height)
        print('Positive Samples:    ', numPos)
        print('Negative Samples:    ', numNeg)
        print('Percent Positive Samples:    ', numPos/(numPos+numNeg))
    else:
        raise Exception(f'Received unexpected input to \'method\' parameter: {method}')

    return df

: 

In [ ]:
BATCH_SIZE = 64
NUM_EPOCHS = 50
H = 128
W = 313
DEVICE = 'gpu'

#load data from dataframe
data_dir = "data/raw_training/training_data/"
target_label = 'murmur_in_recording'
df = (
    ip.loadTrainingData(data_dir)
    .filter(pl.col(target_label) != 'Unknown')
    .pipe(ip.encodeData)
    .select([
        pl.col('audio_file').apply(lambda x: os.path.join(data_dir, x)),
        pl.col(target_label)
    ])
)

df = balanceData(df, 2)

: 

In [ ]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import nvidia.dali.plugin.tf as dali_tf

#prefetch_queue_depth=2*BATCH_SIZE, num_threads=4
@pipeline_def(device_id=0, batch_size=BATCH_SIZE)
def pipe(audioPaths, labels, device, sr=4000, duration=20, augmentData=True):
    # Load audio files
    encoded, labelsOut = fn.readers.file(files=audioPaths, labels=labels)
    audio, _ = fn.decoders.audio(encoded, sample_rate=sr)
    if device=='gpu':
        audio = audio.gpu()
        labelsOut = labelsOut.gpu()

    # Resize samples to uniform length, either by padding it with silence or truncating it
    trimmedSize = duration*sr
    audio = fn.slice(audio, 0, trimmedSize, axes=0, out_of_bounds_policy='pad')

    # Time shift signal to the left or right by a random percent of its original length (max 99%)
    #   NOT IMPLEMENTED 
    # breakpoint = fn.random.uniform(range=(0.,1.))
    # head = fn.slice

    # Get mel spectrogram
    spec = fn.spectrogram(audio, device=device)
    melSpec = fn.mel_filter_bank(spec, sample_rate=sr, nfilter=128)
    melSpec = fn.to_decibels(melSpec, multiplier=10.0)

    if augmentData==True:
        # Apply time and frequency masking
        # maskValue = fn.reductions.mean(melSpec, device='cpu')
        maskValue = 0.
        melSpec = fn.erase(melSpec, axes=0, fill_value=maskValue, anchor=fn.random.uniform(range=(0.,1.)), shape=fn.random.uniform(range=(0.,0.1)), normalized=True, device=device)
        melSpec = fn.erase(melSpec, axes=1, fill_value=maskValue, anchor=fn.random.uniform(range=(0.,1.)), shape=fn.random.uniform(range=(0.,0.1)), normalized=True, device=device)
        normalized = fn.normalize(melSpec)
        samplesOut = normalized
    else:
        samplesOut = melSpec

    return samplesOut, labelsOut


: 

In [ ]:
def splitDataframe(df, split_ratio=0.8):
    total_size = df.height
    head_size = round(split_ratio * total_size)
    tail_size = total_size - head_size

    df = df.sample(frac=1.0, shuffle=True)
    head_df = df.head(head_size)
    tail_df = df.tail(tail_size)
    
    return head_df, tail_df

# Split data into training, validation, and testing sets
train_df, test_df = splitDataframe(df, split_ratio=0.9)
train_df, val_df = splitDataframe(train_df, split_ratio=0.8)
numTrainBatches = math.ceil(train_df.height / BATCH_SIZE)
numValBatches = math.ceil(val_df.height / BATCH_SIZE)
numTestBatches = math.ceil(test_df.height / BATCH_SIZE)
print(f'Training Set Size, Batches:      {train_df.height}, {numTrainBatches}')
print(f'Validation Set Size, Batches:    {val_df.height}, {numValBatches}')
print(f'Testing Set Size, Batches:       {test_df.height}, {numTestBatches}')

# check that sets are unique
for set1, set2 in [[train_df, val_df], [train_df, test_df], [val_df, test_df]]:
    col1 = set1.get_column('audio_file')
    col2 = set2.get_column('audio_file')
    overlap = col1.is_in(col2)
    if overlap.any():
        print(col1.len(), col2.len())
        print(overlap.value_counts())
        raise Exception(
            'Training, validation, and/or testing sets are not unique.' +
            f' Number of overlaps: {col1.filter(overlap).len()}'
        )

with tf.device('/gpu:0' if DEVICE=='gpu' else '/cpu:0'):
    # Define shapes and types of the outputs
    shapes = ((BATCH_SIZE, H, W), (BATCH_SIZE))
    dtypes = (tf.float32, tf.int32)

    # Create pipelines
    train_pipe = pipe(audioPaths=train_df.get_column('audio_file').to_list(),
                    labels=train_df.get_column(target_label).to_list(),
                    device=DEVICE)
    val_pipe = pipe(audioPaths=val_df.get_column('audio_file').to_list(),
                    labels=val_df.get_column(target_label).to_list(),
                    device=DEVICE)            
    test_pipe = pipe(audioPaths=test_df.get_column('audio_file').to_list(),
                    labels=test_df.get_column(target_label).to_list(),
                    device=DEVICE)

    # Create datasets
    kwargs = dict(batch_size=BATCH_SIZE,
                  output_shapes=shapes,
                  output_dtypes=dtypes,
                  device_id=0,
                  prefetch_queue_depth=4,
                  num_threads=4,
                  exec_separated=True)
    train_set = dali_tf.DALIDataset(pipeline=train_pipe, **kwargs)
    val_set = dali_tf.DALIDataset(pipeline=val_pipe, **kwargs)
    test_set = dali_tf.DALIDataset(pipeline=test_pipe, **kwargs)


: 

In [ ]:

# train_pipe.build()
# train_pipe.save_graph_to_dot_file('DALIgraph2.dot', use_colors=True, show_ids=False, show_tensors=True)

: 

In [ ]:
#keras script

import numpy as np
from keras.models import Sequential
from keras import datasets, layers, models

with tf.device('/gpu:0' if DEVICE=='gpu' else '/cpu:0'):
    # Design model
    input_shape = (H, W, 1) # 128,313,1
    model = Sequential()

    # Arcitecture
    # model.add(layers.Conv2D(8, (3,3), activation='relu', input_shape=input_shape, padding='same'))
    # model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    # model.add(layers.Conv2D(16, (5,5), activation='relu'))
    # model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    # model.add(layers.Conv2D(32, (5,5), activation='relu'))
    # model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    # model.add(layers.Conv2D(64, (5,5), activation='relu'))
    # model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    # model.add(layers.Flatten())
    # model.add(layers.Dense(128, activation='relu'))
    # model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dense(1, activation='sigmoid'))



    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Conv2D(128, (3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Dropout(0.2))
    # model.add(layers.Conv2D(256, (3,3), activation='relu'))
    # model.add(layers.MaxPool2D((2,2)))
    # model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1, activation='sigmoid'))



    # model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
    # model.add(layers.Conv2D(16, (3,3), activation='relu'))
    # model.add(layers.MaxPool2D((2,2)))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Conv2D(32, (3,3), activation='relu'))
    # model.add(layers.Conv2D(32, (3,3), activation='relu'))
    # model.add(layers.MaxPool2D((2,2)))
    # model.add(layers.Dropout(0.5))
    # model.add(layers.Flatten())
    # model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dropout(0.5))
    # # model.add(layers.Dense(32, activation='relu'))
    # model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='bce', metrics=['accuracy'])
    model.summary()

    # # Create a TensorBoard callback
    # logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    # tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs, histogram_freq = 1, profile_batch = '10,15')

    # Train model on dataset
    history = model.fit(
        x=train_set,
        validation_data=val_set,
        epochs=NUM_EPOCHS,
        steps_per_epoch=int(numTrainBatches*0.9),
        validation_steps=int(numValBatches*0.9),
        # callbacks=tboard_callback
    )

: 

In [ ]:
#Adapted from Deep Learning with Python by Francois Chollet, 2018
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,NUM_EPOCHS+1)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'bo',label='Training Loss')
ax1.plot(epochs,val_loss_values,'orange', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'bo', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'orange',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()

: 

In [ ]:
model.evaluate(test_set, steps=numTestBatches)

: 

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train', color='red')
plt.plot(history.history['val_loss'], label='test', color='blue')
plt.legend()
plt.show()

: 

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train', color='red')
plt.plot(history.history['val_accuracy'], label='test', color='blue')
plt.legend()
plt.show()

: 

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard


: 

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs

: 